# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow.keras.backend as K
from numpy import mean
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Reading Dataset

In [2]:
df = pd.read_csv("Dataset_all.csv")
df

,Lattice Type,X,Y,Z,Thickness,Force (N),Strain (mm)
0,SplitP,10.0,8.7,9.5,1.1,0.2190,0.0001
1,SplitP,10.0,8.7,9.5,1.1,0.2675,0.0023
2,SplitP,10.0,8.7,9.5,1.1,0.3566,0.0111
3,SplitP,10.0,8.7,9.5,1.1,0.2351,0.0255
4,SplitP,10.0,8.7,9.5,1.1,0.3353,0.0407
...,...,...,...,...,...,...,...
76756,Lidinoid,8.3,7.8,6.1,0.7,1267.1747,11.3756
76757,Lidinoid,8.3,7.8,6.1,0.7,1267.6182,11.3923
76758,Lidinoid,8.3,7.8,6.1,0.7,1268.4763,11.4090
76759,Lidinoid,8.3,7.8,6.1,0.7,1269.4996,11.4256


# Normalize Dataset using RIN

In [3]:
def rank_and_normalize(series):
    ranked_series = series.rank()
    normalized_series = (ranked_series - 0.5) / len(series)
    result = norm.ppf(normalized_series)
    return result

columns_to_normalize = ['X', 'Y', 'Z', 'Thickness', 'Force (N)', 'Strain (mm)']
for column in columns_to_normalize:
    df[column] = rank_and_normalize(df[column])
df

,Lattice Type,X,Y,Z,Thickness,Force (N),Strain (mm)
0,SplitP,2.304713,0.806729,1.516002,0.014924,-2.549112,-3.236146
1,SplitP,2.304713,0.806729,1.516002,0.014924,-2.479119,-2.871088
2,SplitP,2.304713,0.806729,1.516002,0.014924,-2.394080,-2.715482
3,SplitP,2.304713,0.806729,1.516002,0.014924,-2.523859,-2.609514
4,SplitP,2.304713,0.806729,1.516002,0.014924,-2.414051,-2.532233
...,...,...,...,...,...,...,...
76756,Lidinoid,0.208935,0.226942,-0.708049,-0.819751,1.362283,0.907819
76757,Lidinoid,0.208935,0.226942,-0.708049,-0.819751,1.362572,0.911870
76758,Lidinoid,0.208935,0.226942,-0.708049,-0.819751,1.363398,0.915836
76759,Lidinoid,0.208935,0.226942,-0.708049,-0.819751,1.364143,0.919817


# Converting Lattice Type Variable to Numerical

In [4]:
le = preprocessing.LabelEncoder()
df['Lattice Type'] = le.fit_transform(df['Lattice Type'])
df

,Lattice Type,X,Y,Z,Thickness,Force (N),Strain (mm)
0,4,2.304713,0.806729,1.516002,0.014924,-2.549112,-3.236146
1,4,2.304713,0.806729,1.516002,0.014924,-2.479119,-2.871088
2,4,2.304713,0.806729,1.516002,0.014924,-2.394080,-2.715482
3,4,2.304713,0.806729,1.516002,0.014924,-2.523859,-2.609514
4,4,2.304713,0.806729,1.516002,0.014924,-2.414051,-2.532233
...,...,...,...,...,...,...,...
76756,2,0.208935,0.226942,-0.708049,-0.819751,1.362283,0.907819
76757,2,0.208935,0.226942,-0.708049,-0.819751,1.362572,0.911870
76758,2,0.208935,0.226942,-0.708049,-0.819751,1.363398,0.915836
76759,2,0.208935,0.226942,-0.708049,-0.819751,1.364143,0.919817


# Defining Features and Targets

In [5]:
x = df[['Force (N)', 'Strain (mm)']]
y = df[['Lattice Type', 'X', 'Y', 'Z', 'Thickness']]
x

,Force (N),Strain (mm)
0,-2.549112,-3.236146
1,-2.479119,-2.871088
2,-2.394080,-2.715482
3,-2.523859,-2.609514
4,-2.414051,-2.532233
...,...,...
76756,1.362283,0.907819
76757,1.362572,0.911870
76758,1.363398,0.915836
76759,1.364143,0.919817


# Splitting to Train and Test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, shuffle=True)

# Hyperparameters of ANN using GridSearchCV

In [36]:
def r2_score(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

def create_model(activation):
    model = Sequential()
    model.add(Dense(units=90, input_dim=2, activation=activation))
    model.add(Dropout(rate=0.1))
    model.add(Dense(5, activation='tanh'))
    model.compile(loss='mean_squared_error',
                            optimizer='RMSprop',
                            metrics=['MeanSquaredError', r2_score])
    return model


params = {'optimizer__learning_rate': [0.001, 0.01, 0.05],
              'model__activation': ['softmax', 'softsign', 'sigmoid', 'hard_sigmoid'],
              'batch_size': [20, 50],
              'epochs': [10, 20]}

model = KerasRegressor(model=create_model, verbose=1)

num_combinations = 1
for v in params.values():
    num_combinations *= len(v)
print("Number of Combinations: ", num_combinations)

optimal_params = GridSearchCV(estimator=model, param_grid=params, n_jobs=1, cv=5)

result = optimal_params.fit(x, y)

Number of Combinations:  48
Epoch 1/10
3071/3071 [==============================] - 7s 2ms/step - loss: 1.1774 - mean_squared_error: 1.1774 - r2_score: 0.2451
Epoch 2/10
3071/3071 [==============================] - 6s 2ms/step - loss: 1.0956 - mean_squared_error: 1.0956 - r2_score: 0.2970
Epoch 3/10
3071/3071 [==============================] - 7s 2ms/step - loss: 1.0913 - mean_squared_error: 1.0913 - r2_score: 0.3003
Epoch 4/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.0894 - mean_squared_error: 1.0894 - r2_score: 0.3010
Epoch 5/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.0879 - mean_squared_error: 1.0879 - r2_score: 0.3022
Epoch 6/10
3071/3071 [==============================] - 8s 2ms/step - loss: 1.0871 - mean_squared_error: 1.0871 - r2_score: 0.3028
Epoch 7/10
3071/3071 [==============================] - 7s 2ms/step - loss: 1.0861 - mean_squared_error: 1.0861 - r2_score: 0.3030
Epoch 8/10
3071/3071 [==============================] -

768/768 [==============================] - 1s 2ms/step
Epoch 1/10
3071/3071 [==============================] - 9s 3ms/step - loss: 1.2126 - mean_squared_error: 1.2126 - r2_score: 0.2396
Epoch 2/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1157 - mean_squared_error: 1.1157 - r2_score: 0.3005
Epoch 3/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1081 - mean_squared_error: 1.1081 - r2_score: 0.3054
Epoch 4/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1057 - mean_squared_error: 1.1057 - r2_score: 0.3067
Epoch 5/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1047 - mean_squared_error: 1.1047 - r2_score: 0.3075
Epoch 6/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1036 - mean_squared_error: 1.1036 - r2_score: 0.3080
Epoch 7/10
3071/3071 [==============================] - 9s 3ms/step - loss: 1.1030 - mean_squared_error: 1.1030 - r2_score: 0.3081
Epoch 8/10
3071/3071 [======

768/768 [==============================] - 1s 2ms/step
Epoch 1/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1704 - mean_squared_error: 1.1704 - r2_score: 0.2790
Epoch 2/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1486 - mean_squared_error: 1.1486 - r2_score: 0.2924
Epoch 3/10
3071/3071 [==============================] - 8s 2ms/step - loss: 1.1449 - mean_squared_error: 1.1449 - r2_score: 0.2948
Epoch 4/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1438 - mean_squared_error: 1.1438 - r2_score: 0.2958
Epoch 5/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1421 - mean_squared_error: 1.1421 - r2_score: 0.2969
Epoch 6/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1414 - mean_squared_error: 1.1414 - r2_score: 0.2969
Epoch 7/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1399 - mean_squared_error: 1.1399 - r2_score: 0.2978
Epoch 8/10
3071/3071 [======

768/768 [==============================] - 2s 2ms/step
Epoch 1/10
3071/3071 [==============================] - 9s 3ms/step - loss: 1.1619 - mean_squared_error: 1.1619 - r2_score: 0.2940
Epoch 2/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1446 - mean_squared_error: 1.1446 - r2_score: 0.3049
Epoch 3/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1419 - mean_squared_error: 1.1419 - r2_score: 0.3062
Epoch 4/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1412 - mean_squared_error: 1.1412 - r2_score: 0.3069
Epoch 5/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1399 - mean_squared_error: 1.1399 - r2_score: 0.3077
Epoch 6/10
3071/3071 [==============================] - 8s 2ms/step - loss: 1.1392 - mean_squared_error: 1.1392 - r2_score: 0.3084
Epoch 7/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1388 - mean_squared_error: 1.1388 - r2_score: 0.3082
Epoch 8/10
3071/3071 [======

768/768 [==============================] - 1s 2ms/step
Epoch 1/10
3071/3071 [==============================] - 9s 3ms/step - loss: 1.1431 - mean_squared_error: 1.1431 - r2_score: 0.2657
Epoch 2/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1185 - mean_squared_error: 1.1185 - r2_score: 0.2820
Epoch 3/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1116 - mean_squared_error: 1.1116 - r2_score: 0.2865
Epoch 4/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.1048 - mean_squared_error: 1.1048 - r2_score: 0.2911
Epoch 5/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.0996 - mean_squared_error: 1.0996 - r2_score: 0.2942
Epoch 6/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.0971 - mean_squared_error: 1.0971 - r2_score: 0.2961
Epoch 7/10
3071/3071 [==============================] - 8s 3ms/step - loss: 1.0951 - mean_squared_error: 1.0951 - r2_score: 0.2978
Epoch 8/10
3071/3071 [======

...

1229/1229 [==============================] - 4s 3ms/step - loss: 1.1413 - mean_squared_error: 1.1413 - r2_score: 0.3099
Epoch 10/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1398 - mean_squared_error: 1.1398 - r2_score: 0.3109
Epoch 11/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1387 - mean_squared_error: 1.1387 - r2_score: 0.3115
Epoch 12/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1377 - mean_squared_error: 1.1377 - r2_score: 0.3123
Epoch 13/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1372 - mean_squared_error: 1.1372 - r2_score: 0.3124
Epoch 14/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1367 - mean_squared_error: 1.1367 - r2_score: 0.3127
Epoch 15/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1368 - mean_squared_error: 1.1368 - r2_score: 0.3123
Epoch 16/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1357 - me

1229/1229 [==============================] - 4s 3ms/step - loss: 1.0946 - mean_squared_error: 1.0946 - r2_score: 0.3019
Epoch 12/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0938 - mean_squared_error: 1.0938 - r2_score: 0.3025
Epoch 13/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0935 - mean_squared_error: 1.0935 - r2_score: 0.3022
Epoch 14/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0927 - mean_squared_error: 1.0927 - r2_score: 0.3029
Epoch 15/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0921 - mean_squared_error: 1.0921 - r2_score: 0.3032
Epoch 16/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0917 - mean_squared_error: 1.0917 - r2_score: 0.3034
Epoch 17/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0914 - mean_squared_error: 1.0914 - r2_score: 0.3038
Epoch 18/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.0908 - me

1229/1229 [==============================] - 4s 3ms/step - loss: 1.1199 - mean_squared_error: 1.1199 - r2_score: 0.2891
Epoch 14/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1152 - mean_squared_error: 1.1152 - r2_score: 0.2923
Epoch 15/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1104 - mean_squared_error: 1.1104 - r2_score: 0.2953
Epoch 16/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1072 - mean_squared_error: 1.1072 - r2_score: 0.2973
Epoch 17/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1055 - mean_squared_error: 1.1055 - r2_score: 0.2985
Epoch 18/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1041 - mean_squared_error: 1.1041 - r2_score: 0.2992
Epoch 19/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1024 - mean_squared_error: 1.1024 - r2_score: 0.3002
Epoch 20/20
308/308 [==============================] - 1s 2ms/step
Epoch 1/20
1229/1229 

1229/1229 [==============================] - 4s 3ms/step - loss: 1.1179 - mean_squared_error: 1.1179 - r2_score: 0.3020
Epoch 16/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1175 - mean_squared_error: 1.1175 - r2_score: 0.3023
Epoch 17/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1162 - mean_squared_error: 1.1162 - r2_score: 0.3032
Epoch 18/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1158 - mean_squared_error: 1.1158 - r2_score: 0.3032
Epoch 19/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1152 - mean_squared_error: 1.1152 - r2_score: 0.3036
Epoch 20/20
308/308 [==============================] - 1s 2ms/step
Epoch 1/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.2323 - mean_squared_error: 1.2323 - r2_score: 0.2448
Epoch 2/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1839 - mean_squared_error: 1.1839 - r2_score: 0.2744
Epoch 3/20
1229/1229 [=

1229/1229 [==============================] - 4s 3ms/step - loss: 1.1426 - mean_squared_error: 1.1426 - r2_score: 0.2999
Epoch 18/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1410 - mean_squared_error: 1.1410 - r2_score: 0.3008
Epoch 19/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1405 - mean_squared_error: 1.1405 - r2_score: 0.3012
Epoch 20/20
308/308 [==============================] - 1s 2ms/step
Epoch 1/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.2179 - mean_squared_error: 1.2179 - r2_score: 0.2634
Epoch 2/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1727 - mean_squared_error: 1.1727 - r2_score: 0.2908
Epoch 3/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1663 - mean_squared_error: 1.1663 - r2_score: 0.2947
Epoch 4/20
1229/1229 [==============================] - 4s 3ms/step - loss: 1.1632 - mean_squared_error: 1.1632 - r2_score: 0.2966
Epoch 5/20
1229/1229 [===

## Results of GridSearchCV

In [37]:
df_cv_results = pd.DataFrame(optimal_params.cv_results_)
df_cv_results = df_cv_results[['mean_fit_time', 'mean_score_time', 'param_batch_size', 'param_epochs', 'param_model__activation',
                                'param_optimizer__learning_rate', 'mean_test_score', 'rank_test_score']]
df_cv_results.sort_values(by='rank_test_score', inplace=True)
df_cv_results

,mean_fit_time,mean_score_time,param_batch_size,param_epochs,param_model__activation,param_optimizer__learning_rate,mean_test_score,rank_test_score
29,38.175938,0.751064,50,10,softsign,0.05,0.002343,1
7,79.394153,1.656216,20,10,sigmoid,0.01,-0.004419,2
9,81.908190,1.694119,20,10,hard_sigmoid,0.001,-0.005030,3
40,76.336460,0.701359,50,20,softsign,0.01,-0.005487,4
32,39.390995,0.770764,50,10,sigmoid,0.05,-0.005514,5
39,79.501233,0.731050,50,20,softsign,0.001,-0.005518,6
46,81.587862,0.758263,50,20,hard_sigmoid,0.01,-0.005525,7
44,74.667464,0.785693,50,20,sigmoid,0.05,-0.005788,8
25,39.220820,0.771858,50,10,softmax,0.01,-0.005862,9
47,79.923059,0.691967,50,20,hard_sigmoid,0.05,-0.005959,10


## Best Hyperparameters Values

In [38]:
print(optimal_params.best_score_)
print(optimal_params.best_params_)

0.002342556174920557
{'batch_size': 50, 'epochs': 10, 'model__activation': 'softsign', 'optimizer__learning_rate': 0.05}


## Evaluation Metrics of the Best ANN Model

In [43]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

best_model = optimal_params.best_estimator_
y_pred = best_model.predict(X_test)

mse_test = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
mae_test = mean_absolute_error(y_test, y_pred_test)

print("MSE Test:", mse_test)
print("R2 score Test:", r2)
print("RMSE Test:", rmse_test)
print("Mean Absolute Error (MAE) Test:", mae_test)

308/308 [==============================] - 3s 4ms/step
MSE Test: 1.1109401046939875
R2 score Test: 0.058990262038822475
RMSE Test: 0.971894159391517
Mean Absolute Error (MAE) Test: 0.7883884739934914


# Computation Time

In [44]:
fit_time = df_cv_results['mean_fit_time'].sum()
score_time = df_cv_results['mean_score_time'].sum()
mean_time = fit_time + score_time

print("Time to Compute Best Model: {:.2f} hours".format(mean_time / 3600))

Time to Compute Best Model: 1.24 hours
